In [1]:
import torch
from langchain_openai import ChatOpenAI
from langchain.tools import tool
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
import json

In [2]:
def clear_cuda_memory():
    """
    Clear CUDA memory cache to free up GPU resources between queries.
    Only has an effect if CUDA is available.
    """
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
        import gc
        gc.collect()
        print("CUDA memory cache cleared")
    return

In [3]:
clear_cuda_memory()

CUDA memory cache cleared


In [4]:
llm_model = "gpt-oss:20b" #works: qwen3:1.7b, qwen3:latest, hir0rameel/qwen-claude:latest, mistral-small3.2:latest
    #not working (consistently): llama3.2, qwq, qwen3:0.6b, qwen3:14b,qwen3:30b-a3b, deepseek-r1:latest, atombuild/deepseek-r1-claude3.7:14b

In [5]:
llm = ChatOpenAI(
    api_key="ollama",  # dummy key
    model=llm_model,
    base_url="http://localhost:11434/v1",
    temperature=0.1
)

In [6]:
@tool
def get_current_weather(location: str = "Unknown", temperature_format: str = "celsius") -> str:
    """Get the current weather for a specific location. ONLY use this tool when the user explicitly asks about weather conditions, temperature, or weather forecasts for a particular place.
    
    Args:
        location: The location to get weather for
        temperature_format: Temperature format (celsius or fahrenheit)
    """
    return f"Sample weather for {location} in {temperature_format} is 20° and sunny!"

In [7]:
tools = [get_current_weather]

In [8]:
llm_with_tools = llm.bind_tools(tools)

In [9]:
def execute_with_tools(user_input: str, verbose: bool = True):
    """Execute a query with tool support using bound tools"""
    
    if verbose:
        print(f"\n> Entering new chat session...")
        print(f"> User Input: {user_input}")
    
    # Create messages
    messages = [HumanMessage(content=user_input)]
    
    # Get initial response
    response = llm_with_tools.invoke(messages)
    
    if verbose:
        print(f"> AI Response: {response.content}")
    
    # Check if response contains tool calls
    if hasattr(response, 'tool_calls') and response.tool_calls:
        if verbose:
            print(f"> Tool calls detected: {len(response.tool_calls)}")
        
        # Add AI message to conversation
        messages.append(response)
        
        # Execute each tool call
        for tool_call in response.tool_calls:
            tool_name = tool_call['name']
            tool_args = tool_call['args']
            tool_id = tool_call['id']
            
            if verbose:
                print(f"> Calling tool: {tool_name} with args: {tool_args}")
            
            try:
                # FIX: Use tool's invoke method instead of direct call
                if tool_name == "get_current_weather":
                    result = get_current_weather.invoke(tool_args)  # Use .invoke() method
                    
                    if verbose:
                        print(f"> Tool result: {result}")
                    
                    # Add tool result to messages
                    messages.append(ToolMessage(
                        content=result,
                        tool_call_id=tool_id
                    ))
                    
            except Exception as e:
                if verbose:
                    print(f"> Tool execution error: {e}")
                
                # Add error message to conversation
                messages.append(ToolMessage(
                    content=f"Error executing tool: {str(e)}",
                    tool_call_id=tool_id
                ))
        
        # Get final response incorporating tool results
        final_response = llm.invoke(messages)
        
        if verbose:
            print(f"> Final Answer: {final_response.content}")
        
        return {
            "input": user_input,
            "output": final_response.content,
            "intermediate_steps": [(tool_call, "executed") for tool_call in response.tool_calls]
        }
    else:
        # No tools needed, return direct response
        if verbose:
            print(f"> No tools needed, returning direct response")
        
        return {
            "input": user_input,
            "output": response.content,
            "intermediate_steps": []
        }

In [10]:
# Alternative simpler approach without manual message handling
def simple_tool_execution(user_input: str):
    """Simplified tool execution - let OpenAI interface handle everything"""
    response = llm_with_tools.invoke([HumanMessage(content=user_input)])
    
    # If tool calls exist, the response should already include tool execution
    if hasattr(response, 'tool_calls') and response.tool_calls:
        # Process tool calls manually if needed
        messages = [HumanMessage(content=user_input), response]
        
        for tool_call in response.tool_calls:
            if tool_call['name'] == "get_current_weather":
                result = get_current_weather(**tool_call['args'])
                messages.append(ToolMessage(
                    content=result,
                    tool_call_id=tool_call['id']
                ))
        
        # Get final response
        final_response = llm.invoke(messages)
        return final_response.content
    
    return response.content

In [11]:
# Test weather query
for i in range(5):  # Reduced iterations for testing
    print(f"\n=== Weather Test Iteration {i} ===")
    response = execute_with_tools("What's the weather in Aachen, in celsius?", verbose=True)
    print(f"Final response: {response}")
    clear_cuda_memory()


=== Weather Test Iteration 0 ===

> Entering new chat session...
> User Input: What's the weather in Aachen, in celsius?
> AI Response: 
> Tool calls detected: 1
> Calling tool: get_current_weather with args: {'location': 'Aachen', 'temperature_format': 'celsius'}
> Tool result: Sample weather for Aachen in celsius is 20° and sunny!
> Final Answer: 
Final response: {'input': "What's the weather in Aachen, in celsius?", 'output': '', 'intermediate_steps': [({'name': 'get_current_weather', 'args': {'location': 'Aachen', 'temperature_format': 'celsius'}, 'id': 'call_06eij6zy', 'type': 'tool_call'}, 'executed')]}
CUDA memory cache cleared

=== Weather Test Iteration 1 ===

> Entering new chat session...
> User Input: What's the weather in Aachen, in celsius?
> AI Response: 
> Tool calls detected: 1
> Calling tool: get_current_weather with args: {'location': 'Aachen', 'temperature_format': 'celsius'}
> Tool result: Sample weather for Aachen in celsius is 20° and sunny!
> Final Answer: The 

In [12]:
# Alternative: Using the simpler approach
print("\n=== Testing Simple Approach ===")
weather_response = simple_tool_execution("What's the weather in Berlin?")
print(f"Weather response: {weather_response}")


=== Testing Simple Approach ===


/tmp/ipykernel_1418399/2307705365.py:13: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = get_current_weather(**tool_call['args'])


TypeError: BaseTool.__call__() got an unexpected keyword argument 'location'